In [1]:
import pandas as pd
import numpy as np
import textwrap
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv('bbc_text_cls.csv')
df.head()

text    labels
0  Ad sales boost Time Warner profit\n\nQuarterly...  business
1  Dollar gains on Greenspan speech\n\nThe dollar...  business
2  Yukos unit buyer faces loan claim\n\nThe owner...  business
3  High fuel prices hit BA's profits\n\nBritish A...  business
4  Pernod takeover talk lifts Domecq\n\nShares in...  business

In [3]:
# Choose random business article
doc =  df[df.labels == 'business']['text'].sample(random_state=42)

In [4]:
def wrap(x):
    return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)

In [5]:
print(wrap(doc.iloc[0]))

Christmas sales worst since 1981

UK retail sales fell in December,
failing to meet expectations and making it by some counts the worst
Christmas since 1981.

Retail sales dropped by 1% on the month in
December, after a 0.6% rise in November, the Office for National
Statistics (ONS) said.  The ONS revised the annual 2004 rate of growth
down from the 5.9% estimated in November to 3.2%. A number of
retailers have already reported poor figures for December.  Clothing
retailers and non-specialist stores were the worst hit with only
internet retailers showing any significant growth, according to the
ONS.

The last time retailers endured a tougher Christmas was 23 years
previously, when sales plunged 1.7%.

The ONS echoed an earlier
caution from Bank of England governor Mervyn King not to read too much
into the poor December figures.  Some analysts put a positive gloss on
the figures, pointing out that the non-seasonally-adjusted figures
showed a performance comparable with 2003. The Novembe

In [6]:
# Excludes the title from sentences tokenization. 
sents = nltk.sent_tokenize(doc.iloc[0].split('\n', 1)[1])

In [7]:
# nltk.download('stopwords')

In [8]:
featurizer = TfidfVectorizer(stop_words=stopwords.words('english'))


In [9]:
X = featurizer.fit_transform(sents)

In [10]:
def get_sentence_score(tfidf_row):
    # return the average of the non-zero values
    # of the tf-idf vector representation of a sentence
    x = tfidf_row[tfidf_row != 0]
    return x.mean()

In [11]:
scores = np.zeros(len(sents))
for i in range(len(sents)):
    score = get_sentence_score(X[i, :])
    scores[i] = score

In [12]:
# Exercise: can you think of a way to compute all the scores in one step?

In [13]:
sort_idx = np.argsort(-scores)
sort_idx

array([ 3,  2, 10, 16, 12,  5, 11,  1,  7,  4,  6, 14,  9,  0,  8, 13, 15],
      dtype=int64)

In [14]:
print("Generated summary:")
for i in sort_idx[:5]:
    print(wrap("%.2f: %s" % (scores[i], sents[i])))

Generated summary:
0.37: A number of retailers have already reported poor figures for
December.
0.35: The ONS revised the annual 2004 rate of growth down from the
5.9% estimated in November to 3.2%.
0.35: However, reports from some High Street retailers highlight the
weakness of the sector.
0.31: "Our view is the Bank of England will keep its powder dry and
wait to see the big picture."
0.31: And a British Retail Consortium survey found that Christmas 2004
was the worst for 10 years.


In [15]:
# Getting the title:
doc.iloc[0].split("\n", 1)[0]

'Christmas sales worst since 1981'

In [16]:
def summarize(text):
    sents = nltk.sent_tokenize(text)
    
    # perform tf-idf
    X = featurizer.fit_transform(sents)
    
    # compute scores for each sentences
    scores = np.zeros(len(sents))
    for i in range(len(sents)):
        score = get_sentence_score(X[i, :])
        scores[i] = score
    
    # sort the scores
    sort_idx = np.argsort(-scores)
    
    # print summary
    for i in sort_idx[:5]:
        print(wrap("%.2f: %s" % (scores[i], sents[i])))
        

In [17]:
doc = df[df.labels == 'entertainment']['text'].sample(random_state=123)
summarize(doc.iloc[0].split("\n", 1)[1])

0.32: The Black Eyed Peas won awards for best R 'n' B video and
sexiest video, both for Hey Mama.
0.32: The ceremony was held at the Luna Park fairground in Sydney
Harbour and was hosted by the Osbourne family.
0.31: Goodrem, Green Day and the Black Eyed Peas took home two awards
each.
0.31: Other winners included Green Day, voted best group, and the
Black Eyed Peas.
0.31: The VH1 First Music Award went to Cher honouring her
achievements within the music industry.


In [18]:
# Getting the title:
doc.iloc[0].split("\n", 1)[0]

'Goodrem wins top female MTV prize'

In [19]:
print(wrap(doc.iloc[0]))

Goodrem wins top female MTV prize

Pop singer Delta Goodrem has
scooped one of the top individual prizes at the first Australian MTV
Music Awards.

The 21-year-old singer won the award for best female
artist, with Australian Idol runner-up Shannon Noll taking the title
of best male at the ceremony.  Goodrem, known in both Britain and
Australia for her role as Nina Tucker in TV soap Neighbours, also
performed a duet with boyfriend Brian McFadden.  Other winners
included Green Day, voted best group, and the Black Eyed Peas.
Goodrem, Green Day and the Black Eyed Peas took home two awards each.
As well as best female, Goodrem also took home the Pepsi Viewers
Choice Award, whilst Green Day bagged the prize for best rock video
for American Idiot.  The Black Eyed Peas won awards for best R 'n' B
video and sexiest video, both for Hey Mama.  Local singer and
songwriter Missy Higgins took the title of breakthrough artist of the
year, with Australian Idol winner Guy Sebastian taking the honours f

## Text summarization using TextRank

In [20]:
X = featurizer.fit_transform(sents)

In [21]:
# Compute cosine similarity matrix
from sklearn.metrics.pairwise import cosine_similarity
S = cosine_similarity(X)

In [22]:
S.shape

(17, 17)

In [23]:
len(sents)

17

In [24]:
# Normalize similarity matrix
S /= S.sum(axis=1, keepdims=True)

In [25]:
S[4].sum()

1.0

In [26]:
# Uniform transition matrix
U = np.ones_like(S)/len(S)

In [27]:
U[0].sum()

1.0

In [28]:
# Smooth similarity Matrix
factor = 0.15
S = (1- factor)* S + factor * U

In [29]:
S[0].sum()

1.0

In [30]:
# find the limiting / Stationary distributions
eigenvals, eigenvecs = np.linalg.eig(S.T)

In [31]:
# Theory says one of the eigen values must be one.
eigenvals

array([1.        , 0.24245466, 0.72108199, 0.67644122, 0.34790129,
       0.34417302, 0.3866884 , 0.40333562, 0.41608572, 0.44238593,
       0.63909999, 0.62556792, 0.58922572, 0.57452382, 0.48511399,
       0.51329157, 0.52975372])

In [32]:
eigenvecs[:,0]

array([-0.24206557, -0.27051337, -0.2213806 , -0.28613638, -0.25065894,
       -0.2499217 , -0.279622  , -0.21515455, -0.2226665 , -0.22745415,
       -0.2059112 , -0.20959727, -0.23526242, -0.24203809, -0.23663025,
       -0.2940483 , -0.20865607])

In [33]:
eigenvecs[:,0].dot(S)

array([-0.24206557, -0.27051337, -0.2213806 , -0.28613638, -0.25065894,
       -0.2499217 , -0.279622  , -0.21515455, -0.2226665 , -0.22745415,
       -0.2059112 , -0.20959727, -0.23526242, -0.24203809, -0.23663025,
       -0.2940483 , -0.20865607])

In [34]:
eigenvecs[:, 0] / eigenvecs[: ,0].sum()

array([0.05907327, 0.06601563, 0.05402535, 0.06982824, 0.06117038,
       0.06099047, 0.06823848, 0.05250595, 0.05433915, 0.05550753,
       0.05025022, 0.05114976, 0.05741304, 0.05906657, 0.05774684,
       0.07175905, 0.05092007])

In [35]:
limiting_dist = np.ones(len(S))/len(S)
threshold = 1e-8
delta = float('inf')
iters  = 0
while delta > threshold:
    iters += 1
    
    # Markov transition
    p = limiting_dist.dot(S)
    
    # Compute change in limiting distribution
    delta = np.abs(p-limiting_dist).sum()
    
    # update limiting distribution
    limiting_dist = p
print(iters)

41


In [36]:
limiting_dist

array([0.05907327, 0.06601563, 0.05402534, 0.06982824, 0.06117038,
       0.06099047, 0.06823848, 0.05250595, 0.05433915, 0.05550753,
       0.05025022, 0.05114977, 0.05741304, 0.05906657, 0.05774685,
       0.07175905, 0.05092008])

In [37]:
limiting_dist.sum()

0.9999999999999988

In [39]:
np.abs(eigenvecs[:, 0] / eigenvecs[:, 0].sum() - limiting_dist).sum()

1.996473900783835e-08

In [40]:
sort_idx = np.argsort(-scores)

In [41]:
# Printing top 5 sentences.
print("Generated summary: ")
for i in sort_idx[:5]:
    print(wrap("%.2f: %s" % (scores[i], sents[i])))

Generated summary: 
0.37: A number of retailers have already reported poor figures for
December.
0.35: The ONS revised the annual 2004 rate of growth down from the
5.9% estimated in November to 3.2%.
0.35: However, reports from some High Street retailers highlight the
weakness of the sector.
0.31: "Our view is the Bank of England will keep its powder dry and
wait to see the big picture."
0.31: And a British Retail Consortium survey found that Christmas 2004
was the worst for 10 years.


In [42]:
doc.iloc[0].split("\n")[0]

'Goodrem wins top female MTV prize'